In [1]:
from pathlib import Path

from deepjr.inference import reset_random_seeds, estimate_parameters

2024-06-15 20:32:21.976081: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [13]:
import os
import numpy as np
import tensorflow as tf
from sklearn.model_selection import train_test_split
from tensorflow.keras.layers import LSTM, Dense, Bidirectional
from tensorflow.keras.models import Sequential
from tensorflow.keras import initializers
import pickle
import xarray as xr
import pandas as pd
import random as python_random
from tensorflow.keras.callbacks import EarlyStopping

from sklearn.preprocessing import MinMaxScaler
import scipy.stats


def reset_random_seeds(seed_value=1234):
    ''' Set random seeds for reproducibility. '''
    os.environ['PYTHONHASHSEED'] = str(seed_value)
    python_random.seed(seed_value)
    np.random.seed(seed_value)
    tf.random.set_seed(seed_value)
    # Configure TensorFlow settings if necessary (rarely needed)
    # tf.config.threading.set_intra_op_parallelism_threads(1)
    # tf.config.threading.set_inter_op_parallelism_threads(1)


def create_bi_lstm_model(input_shape, lstm_units=64, dropout_rate=0.1, 
                         dense_units=8):
    ''' Function to create the LSTM model. '''
    model = Sequential()

    # First layer, needs to return sequences for subsequent layers
    lstm_model = LSTM(lstm_units, return_sequences=False, dropout=dropout_rate,
                      kernel_initializer=initializers.GlorotUniform(seed=4287),
                      bias_initializer=initializers.Constant(0.001))
    bilstm_model = Bidirectional(lstm_model, input_shape=input_shape)
    model.add(bilstm_model)

    # Final output layer
    model.add(Dense(dense_units, activation='linear',
                    kernel_initializer=initializers.GlorotUniform(seed=4287),
                    bias_initializer=initializers.Constant(0.001)))

    return model


def train_model(model, X_train, y_train, X_val, y_val, epochs,
                batch_size):
    ''' Function to compile and train the model. '''
    early_stop = EarlyStopping( monitor='val_loss',    # Monitor validation loss
                                min_delta=0.01,       # an improvement significant if it's greater than 0.001
                                patience=50,           # Number of epochs to wait after the last improvement
                                verbose=1,             # Print messages when stopping
                                mode='min',            # Stop training when the quantity monitored has stopped decreasing
                                restore_best_weights=True) # Restore model weights from the epoch with the best value of the monitored quantity.)
    
    model.compile(optimizer='adam', loss="mse", # loss=snr_inv_loss_db, 
                  metrics=['mse'])
    history = model.fit(X_train, y_train, epochs=epochs,
                        batch_size=batch_size, callbacks=[early_stop],
                        validation_data=(X_val, y_val))
    return history


def snr_inv_loss_db(y_true, y_pred):
    # Calculate the signal power (mean squared value of the true signal)
    signal_power = tf.reduce_mean(tf.square(y_true))

    # Calculate the noise power (mean squared error)
    noise_power = tf.reduce_mean(tf.square(y_true - y_pred))

    # Calculate the inverse SNR
    snr_inv = noise_power / signal_power  # Inverse SNR
    
    # Convert inverse SNR to decibels
    snr_inv_db = 10 * tf.math.log(snr_inv) / tf.math.log(10.0) 

    tf.debugging.Assert(not tf.math.is_nan(snr_inv_db), [snr_inv_db, noise_power, 
                                                         y_pred, tf.shape(y_pred), 
                                                         tf.math.reduce_sum(tf.cast(tf.math.is_nan(y_pred), tf.int16))])
    tf.debugging.Assert(not tf.math.is_inf(snr_inv_db), [snr_inv_db, signal_power, noise_power, y_true, y_pred])

    return snr_inv_db


def get_non_outliers_sim_no(x):
    x = np.abs(x).mean(dim=["time", "ch_names"])

    q1, q3 = x.quantile([0.25, 0.75])

    th_min = q1 - 3*(q3-q1)
    th_max = q3 + 3*(q3-q1)

    return x[(x>th_min) & (x<th_max)].sim_no


def estimate_parameters(parameter, noise, input_path, pred_path,
                        results_file_path, estimated_parameters,
                        nb_simulations=1000,
                        epochs=150, batch_size=32,
                        liftby=10):

    # Load the dataset
    noise_str = str(noise).replace(".", "_")
    fname = f'xarr_noise_{parameter}_{nb_simulations}_{noise_str}.nc'
    dataset = xr.open_dataset(input_path / fname)
    dataset = dataset.dropna("sim_no")

    # Drop outliers
    non_outliers_no = get_non_outliers_sim_no(dataset['evoked'])
    dataset = dataset.sel(sim_no=non_outliers_no)

    X = dataset["evoked"].transpose("sim_no", "time", "ch_names")
    X = X.squeeze().to_numpy()
    y = dataset["parameters"].sel(param=estimated_parameters)


    scaler = MinMaxScaler()

    # Scale the data: Fit and transform the scaler to only the specified parameter
    y_scaled = scaler.fit_transform(y)  

    # Create a DataFrame for the scaled data
    # y_scaled_df = pd.DataFrame(y_scaled)
        
    lift_scale = 10**liftby     # for avoiding gradient descent vanishing problem 

    #testing and training split
    X_train, X_test, y_train, y_test = train_test_split(X*lift_scale,
                                                        y_scaled, test_size=0.10,
                                                        random_state=68)
    
    #for validation split
    X_train, x_val, y_train, y_val = train_test_split(X_train, y_train,
                                                      test_size=0.10,
                                                      random_state=68)
    
    # Create the LSTM model
    model = create_bi_lstm_model(input_shape=(X_train.shape[1], X_train.shape[2]))

    # Train the model
    train_model(model, X_train, y_train, X_test, y_test, epochs=epochs, 
                batch_size=batch_size)
    
    # Evaluate the model
    loss_mse_results = model.evaluate(x_val, y_val, batch_size=batch_size)
    
    print(f'Results for {parameter}: Loss = {loss_mse_results[0]}, MSE = {loss_mse_results[1]}')
    
    # Generate predictions
    predictions = model.predict(x_val)
    predictions_original_scale = scaler.inverse_transform(predictions)

    y_val_original = scaler.inverse_transform(y_val)

    #save predictions for future debugging
    df_pred = pd.DataFrame()
    df_pred = pd.DataFrame(predictions_original_scale, columns=estimated_parameters)
    df_orig = pd.DataFrame(y_val_original, columns=estimated_parameters)
    # Convert predictions to a DataFrame

    # Save the DataFrame to a CSV file
    df_pred.to_csv(f'{pred_path}predictions_{parameter}.csv', index=False)
    df_orig .to_csv(f'{pred_path}orignal_{parameter}.csv', index=False)

    # Assuming df_pred and df_orig have the same columns for which you want to compute correlations
    column_correlations = {}

    for column in df_pred.columns:  # Loop through each column name
        # Calculate Spearman correlation for each column
        correlation, p_value = scipy.stats.spearmanr(df_pred[column], df_orig[column])
        
        # Prepare and save the results
        result_string = f"Correlation for {column}: r = {correlation:.3f}, p-value = {p_value:.3f}\n"
        column_correlations[column] = result_string

    with open(results_file_path, 'a') as file:    

        # You can now print or write these results to a file
        for column, result in column_correlations.items():
            
            result_string = f"-- For {noise}"

            file.write(result_string)
            file.write(result)    
    
    model.save(f"deepjr_{parameter}_{nb_simulations}_{noise_str}.keras")

    return model


In [14]:
# Column names
colmuns_to_call = ['all'] # predicting all parameters together
estimated_parameters = ['A_e', 'A_i', 'b_e', 'b_i', 'a_1', 'a_2', 'a_3', 'a_4']
noise_fact = [1.0]
nb_simulations = 1000

base_path = Path('deepjr_training_data')
base_path.mkdir(exist_ok=True)

pred_path = base_path / 'predictions'
pred_path.mkdir(exist_ok=True)

#save the results in the path
results_file_path = pred_path / "correlation_results.txt"

    # Loop over each parameter
for parameter in colmuns_to_call :
    for noise in noise_fact:
        reset_random_seeds()  # Reset the seeds

        import warnings
        with warnings.catch_warnings():
            #warnings.filterwarnings('error')
            estimate_parameters(parameter, noise, base_path, pred_path, results_file_path, estimated_parameters,
                            nb_simulations=nb_simulations)


Epoch 1/150
25/25 [==============================] - 21s 665ms/step - loss: 0.2417 - mse: 0.2417 - val_loss: 0.0567 - val_mse: 0.0567
Epoch 2/150
25/25 [==============================] - 16s 657ms/step - loss: 0.0788 - mse: 0.0788 - val_loss: 0.0347 - val_mse: 0.0347
Epoch 3/150
25/25 [==============================] - 18s 711ms/step - loss: 0.0469 - mse: 0.0469 - val_loss: 0.0260 - val_mse: 0.0260
Epoch 4/150
25/25 [==============================] - 17s 676ms/step - loss: 0.0385 - mse: 0.0385 - val_loss: 0.0282 - val_mse: 0.0282
Epoch 5/150
25/25 [==============================] - 19s 770ms/step - loss: 0.0320 - mse: 0.0320 - val_loss: 0.0253 - val_mse: 0.0253
Epoch 6/150
25/25 [==============================] - 18s 737ms/step - loss: 0.0296 - mse: 0.0296 - val_loss: 0.0247 - val_mse: 0.0247
Epoch 7/150
25/25 [==============================] - 16s 636ms/step - loss: 0.0285 - mse: 0.0285 - val_loss: 0.0248 - val_mse: 0.0248
Epoch 8/150
25/25 [==============================] - 18s 719ms

/var/folders/3k/220tdhsn33709gq9ylr8c6kh0000gp/T/ipykernel_36054/3159543713.py:173: ConstantInputWarning: An input array is constant; the correlation coefficient is not defined.
  correlation, p_value = scipy.stats.spearmanr(df_pred[column], df_orig[column])


In [17]:
model = tf.keras.models.load_model(f'deepjr_all_1000_1_0.keras')

In [33]:
pwd

'/Users/christian/Code/Jansen-Rit-Model-Benchmarking-Deep-Learning/notebooks'

In [22]:
noise_str = str(noise).replace(".", "_")
fname = f'xarr_noise_all_1000_1_0.nc'
dataset = xr.open_dataset(base_path / fname)
dataset = dataset.dropna("sim_no")

X = dataset["evoked"].transpose("sim_no", "time", "ch_names")
X = X.squeeze()
X

<xarray.DataArray 'evoked' (sim_no: 996, time: 1201, ch_names: 64)>
array([[[-1.90674472e-08,  2.00061125e-09,  2.05262915e-08, ...,
          3.12728358e-08, -1.18020206e-08,  3.17991839e-08],
        [ 6.55893789e-09, -1.74803490e-08, -3.45928175e-08, ...,
         -2.73968872e-08,  1.11450039e-09, -1.55213049e-08],
        [ 1.74994155e-08, -1.47419563e-08, -8.31142452e-09, ...,
          1.34164869e-08,  1.05742995e-08, -7.53134719e-09],
        ...,
        [-1.59821743e-08, -4.39752961e-08,  2.45047115e-08, ...,
         -5.94876043e-09, -4.48635026e-09, -8.52139993e-09],
        [-1.22215303e-08, -8.62962984e-09,  5.44828210e-10, ...,
         -1.38403148e-08, -3.40392349e-08,  2.95794290e-08],
        [ 2.74510178e-08, -2.27165806e-08, -1.60641122e-08, ...,
          3.82301527e-08, -1.64155992e-08, -7.03687425e-09]],

       [[-1.90674472e-08,  2.00061125e-09,  2.05262915e-08, ...,
          3.12728358e-08, -1.18020206e-08,  3.17991839e-08],
        [ 6.55893789e-09, -1.74803490e-08, -3.45928175e-08, ...,
         -2.73968872e-08,  1.11450039e-09, -1.55213049e-08],
        [ 1.74994155e-08, -1.47419563e-08, -8.31142452e-09, ...,
          1.34164869e-08,  1.05742995e-08, -7.53134719e-09],
...
        [-1.59821743e-08, -4.39752961e-08,  2.45047115e-08, ...,
         -5.94876043e-09, -4.48635026e-09, -8.52139993e-09],
        [-1.22215303e-08, -8.62962984e-09,  5.44828211e-10, ...,
         -1.38403148e-08, -3.40392349e-08,  2.95794290e-08],
        [ 2.74510178e-08, -2.27165806e-08, -1.60641122e-08, ...,
          3.82301527e-08, -1.64155992e-08, -7.03687425e-09]],

       [[ 1.30377873e-07,  5.51005525e-07,  3.99300571e-07, ...,
         -1.72040830e-07, -2.17386009e-07, -1.23150201e-07],
        [ 1.54521982e-07,  5.26079249e-07,  3.40424582e-07, ...,
         -2.28693982e-07, -2.02430399e-07, -1.68933822e-07],
        [ 1.63979999e-07,  5.23371650e-07,  3.62948628e-07, ...,
         -1.85863787e-07, -1.90931258e-07, -1.59406805e-07],
        ...,
        [-1.77875137e-06, -6.51971470e-06, -4.44329415e-06, ...,
          2.39221979e-06,  2.42046159e-06,  1.81917051e-06],
        [-1.77670617e-06, -6.49067092e-06, -4.47160188e-06, ...,
          2.38666202e-06,  2.39326855e-06,  1.85904996e-06],
        [-1.73874927e-06, -6.51106052e-06, -4.49255920e-06, ...,
          2.44106655e-06,  2.41325232e-06,  1.82421250e-06]]])
Coordinates:
  * sim_no    (sim_no) int64 0 1 2 3 4 5 6 7 ... 992 993 994 995 996 997 998 999
  * ch_names  (ch_names) object 'Fp1' 'AF7' 'AF3' 'F1' ... 'PO8' 'PO4' 'O2'
  * time      (time) float64 -0.2 -0.199 -0.198 -0.197 ... 0.997 0.998 0.999 1.0

array([[[-1.90674472e-08,  2.00061122e-09,  2.05262915e-08, ...,
          3.12728358e-08, -1.18020206e-08,  3.17991839e-08],
        [ 6.55893788e-09, -1.74803490e-08, -3.45928175e-08, ...,
         -2.73968871e-08,  1.11450040e-09, -1.55213049e-08],
        [ 1.74994155e-08, -1.47419564e-08, -8.31142453e-09, ...,
          1.34164869e-08,  1.05742995e-08, -7.53134718e-09],
        ...,
        [-1.59821743e-08, -4.39752961e-08,  2.45047115e-08, ...,
         -5.94876043e-09, -4.48635026e-09, -8.52139993e-09],
        [-1.22215303e-08, -8.62962984e-09,  5.44828213e-10, ...,
         -1.38403148e-08, -3.40392349e-08,  2.95794290e-08],
        [ 2.74510178e-08, -2.27165806e-08, -1.60641122e-08, ...,
          3.82301527e-08, -1.64155992e-08, -7.03687425e-09]]])

In [32]:
model.predict(np.expand_dims(X.sel(sim_no=10).values, 0))

1/1 [==============================] - 1s 952ms/step


array([[0.02514719, 0.01293286, 0.00265106, 0.01654259, 0.02244885,
        0.01212153, 0.03041763, 0.00209727]], dtype=float32)

array([[-1.90674472e-08,  2.00061122e-09,  2.05262915e-08, ...,
         3.12728358e-08, -1.18020206e-08,  3.17991839e-08],
       [ 6.55893788e-09, -1.74803490e-08, -3.45928175e-08, ...,
        -2.73968871e-08,  1.11450040e-09, -1.55213049e-08],
       [ 1.74994155e-08, -1.47419564e-08, -8.31142453e-09, ...,
         1.34164869e-08,  1.05742995e-08, -7.53134718e-09],
       ...,
       [-1.59821743e-08, -4.39752961e-08,  2.45047115e-08, ...,
        -5.94876043e-09, -4.48635026e-09, -8.52139993e-09],
       [-1.22215303e-08, -8.62962984e-09,  5.44828213e-10, ...,
        -1.38403148e-08, -3.40392349e-08,  2.95794290e-08],
       [ 2.74510178e-08, -2.27165806e-08, -1.60641122e-08, ...,
         3.82301527e-08, -1.64155992e-08, -7.03687425e-09]])